In [2]:
import os
import random
import json
import csv
from faker import Faker
import subprocess
import sys

def instalar_dependencias():
    """Instala las dependencias necesarias si no están disponibles"""
    try:
        import faker
    except ImportError:
        print("Instalando la librería Faker...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "faker"])
        global Faker
        from faker import Faker

def crear_carpeta_datos():
    """Crea la carpeta para almacenar los datos si no existe"""
    carpeta = "data_dulcino"
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
        print(f"Carpeta '{carpeta}' creada exitosamente.")
    else:
        print(f"La carpeta '{carpeta}' ya existe.")
    return carpeta

def generar_data_sintetica(carpeta):
    """Genera datos sintéticos válidos e inválidos para pruebas"""
    fake = Faker('es_ES')
    
    # Configuraciones según requisitos
    CATEGORIAS_VALIDAS = ['Chocolates', 'Caramelos', 'Mashmelos', 'Galletas', 'Salados', 'Gomas de mascar']
    ESTADOS_VENTA = ['Si', 'No']
    NUM_PRODUCTOS = 100  # 70% válidos, 30% inválidos
    
    productos = []
    
    # Generar productos válidos (70%)
    for _ in range(int(NUM_PRODUCTOS * 0.7)):
        # Nombre válido (<= 20 caracteres)
        nombre = fake.unique.word().capitalize()[:20]
        
        # Precio válido (0 < precio < 999)
        precio = round(random.uniform(0.5, 998.99), 2)
        
        # 1-3 categorías válidas
        num_categorias = random.randint(1, 3)
        categorias = random.sample(CATEGORIAS_VALIDAS, num_categorias)
        
        # Estado de venta válido
        en_venta = random.choice(ESTADOS_VENTA)
        
        productos.append({
            'tipo': 'valido',
            'nombre': nombre,
            'precio': precio,
            'categorias': categorias,
            'en_venta': en_venta
        })
    
    # Generar productos inválidos (30%)
    for _ in range(int(NUM_PRODUCTOS * 0.3)):
        tipo_error = random.choice(['nombre', 'precio', 'categorias', 'estado_venta'])
        
        if tipo_error == 'nombre':
            # Nombre inválido (> 20 caracteres)
            nombre = fake.text(max_nb_chars=50)[:50]
            precio = round(random.uniform(0.5, 998.99), 2)
            num_categorias = random.randint(1, 3)
            categorias = random.sample(CATEGORIAS_VALIDAS, num_categorias)
            en_venta = random.choice(ESTADOS_VENTA)
            
        elif tipo_error == 'precio':
            # Precio inválido (texto o fuera de rango)
            nombre = fake.unique.word().capitalize()[:20]
            if random.random() > 0.5:
                precio = fake.word()  # Texto como precio
            else:
                precio = round(random.choice([
                    random.uniform(-100, 0),
                    random.uniform(999, 1500)
                ]), 2)
            num_categorias = random.randint(1, 3)
            categorias = random.sample(CATEGORIAS_VALIDAS, num_categorias)
            en_venta = random.choice(ESTADOS_VENTA)
            
        elif tipo_error == 'categorias':
            # Categorías inválidas
            nombre = fake.unique.word().capitalize()[:20]
            precio = round(random.uniform(0.5, 998.99), 2)
            categorias = [fake.word() for _ in range(random.randint(1, 3))]  # Categorías falsas
            en_venta = random.choice(ESTADOS_VENTA)
            
        else:  # estado_venta
            # Estado de venta inválido
            nombre = fake.unique.word().capitalize()[:20]
            precio = round(random.uniform(0.5, 998.99), 2)
            num_categorias = random.randint(1, 3)
            categorias = random.sample(CATEGORIAS_VALIDAS, num_categorias)
            en_venta = fake.word()  # Estado inválido
        
        productos.append({
            'tipo': f'invalido_{tipo_error}',
            'nombre': nombre,
            'precio': precio,
            'categorias': categorias,
            'en_venta': en_venta
        })
    
    # Mezclar productos
    random.shuffle(productos)
    
    # Guardar en JSON
    with open(os.path.join(carpeta, 'productos.json'), 'w', encoding='utf-8') as f:
        json.dump(productos, f, ensure_ascii=False, indent=2)
    
    # Guardar en CSV (para fácil visualización)
    with open(os.path.join(carpeta, 'productos.csv'), 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['tipo', 'nombre', 'precio', 'categorias', 'en_venta'])
        for p in productos:
            writer.writerow([
                p['tipo'],
                p['nombre'],
                p['precio'],
                '|'.join(p['categorias']),
                p['en_venta']
            ])
    
    print(f"\n✅ Datos generados exitosamente en la carpeta '{carpeta}'")
    print(f"- Total productos: {NUM_PRODUCTOS}")
    print(f"- Productos válidos: {int(NUM_PRODUCTOS * 0.7)}")
    print(f"- Productos inválidos: {int(NUM_PRODUCTOS * 0.3)}")
    print("  - Tipos de errores: nombre, precio, categorias, estado_venta")
    print("- Archivos generados:")
    print(f"  - productos.json (formato completo)")
    print(f"  - productos.csv (formato tabular)")

if __name__ == "__main__":
    print("=== Generador de Data Sintética para Confitería Dulcino ===")
    print("Instalando dependencias si es necesario...")
    instalar_dependencias()
    
    print("\nCreando estructura de carpetas...")
    carpeta_datos = crear_carpeta_datos()
    
    print("\nGenerando datos sintéticos...")
    generar_data_sintetica(carpeta_datos)

=== Generador de Data Sintética para Confitería Dulcino ===
Instalando dependencias si es necesario...

Creando estructura de carpetas...
Carpeta 'data_dulcino' creada exitosamente.

Generando datos sintéticos...

✅ Datos generados exitosamente en la carpeta 'data_dulcino'
- Total productos: 100
- Productos válidos: 70
- Productos inválidos: 30
  - Tipos de errores: nombre, precio, categorias, estado_venta
- Archivos generados:
  - productos.json (formato completo)
  - productos.csv (formato tabular)
